In [4]:
import pandas as pd
from gensim.models.ldamodel import LdaModel
from gensim.corpora import Dictionary

/home/eric/.local/lib/python3.5/site-packages/scipy/sparse/sparsetools.py:20: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [5]:
train_df = pd.read_csv('../dataset/preprocessed_train.csv')
test_df = pd.read_csv('../dataset/preprocessed_test.csv')

In [6]:
corpus = pd.concat([train_df.comment_text, test_df.comment_text]).reset_index(drop=True).dropna().astype(str)

In [7]:
corpus_split = corpus.str.split()

In [8]:
from gensim import corpora, models

dictionary = corpora.Dictionary(corpus_split)

In [9]:
corpus_BoW = [dictionary.doc2bow(text) for text in corpus_split]


In [10]:
lda = LdaModel(corpus_BoW, id2word=dictionary, num_topics=50)

In [11]:
import pyLDAvis

In [34]:
topics = {topic_id: words for (topic_id, words) in lda.print_topics(num_topics=50)}

In [81]:
topics.keys()

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49])

In [37]:
for topic, p in lda.get_document_topics(training_BoW[0]):
    print(topics[topic])
    print()

0.085*"edits" + 0.042*"warning" + 0.035*"constructive" + 0.026*"vandalism" + 0.024*"user" + 0.021*"reverted" + 0.020*"block" + 0.019*"making" + 0.019*"remove" + 0.018*"report"

0.030*"utc" + 0.028*"th" + 0.021*"july" + 0.020*"june" + 0.020*"january" + 0.020*"august" + 0.020*"march" + 0.019*"september" + 0.018*"october" + 0.018*"december"

0.108*"externalink" + 0.081*"penis" + 0.079*"externallink" + 0.028*"email" + 0.025*"fellow" + 0.025*"apologies" + 0.021*"best" + 0.020*"reviewing" + 0.020*"checked" + 0.019*"set"

0.050*"dont" + 0.037*"im" + 0.020*"youre" + 0.019*"want" + 0.017*"right" + 0.015*"thats" + 0.015*"going" + 0.014*"said" + 0.012*"things" + 0.012*"wrong"



In [38]:
for topic, p in lda.get_document_topics(training_BoW[1]):
    print(topics[topic])
    print()

0.077*"editing" + 0.052*"continue" + 0.050*"ip" + 0.046*"address" + 0.036*"unconstructive" + 0.034*"pages" + 0.031*"blocked" + 0.029*"account" + 0.026*"stop" + 0.024*"didnt"

0.082*"links" + 0.076*"link" + 0.040*"automated" + 0.036*"external" + 0.020*"added" + 0.015*"add" + 0.015*"legitimate" + 0.014*"greek" + 0.014*"bots" + 0.013*"information"



In [39]:
train_df.comment_text[0]

'nonsense kiss geek said true ill account terminated'

In [40]:
train_df.comment_text[1]

'vandalize pages w merwin continue blocked editing'

In [41]:
dictionary.save('../lda_models/lda_1/dictionary.d')

In [42]:
lda.save('../lda_models/lda_1/lda.lda')

## Get LDA Features

In [82]:
from keras.preprocessing.sequence import pad_sequences

def get_lda_features(lda_model, texts, maxlen):    
    """
    Add one to the lda topic for embedding purposes as padding is 0 we do not want to 
    represent padding as a topic embedding
    """
    return pad_sequences(list(map(lambda text: 
                         list(map(lambda x: x[0] + 1,
                                  lda_model.get_document_topics(text))), texts)), maxlen=maxlen)
    

In [83]:
training_BoW = map(lambda text: dictionary.doc2bow(text), train_df.comment_text.astype(str).str.split())
test_BoW = map(lambda text: dictionary.doc2bow(text), test_df.comment_text.astype(str).str.split())

In [84]:
train_lda_matrix = get_lda_features(lda, training_BoW, 10)

In [85]:
test_lda_matrix = get_lda_features(lda, test_BoW, 10)

In [86]:
import numpy as np

np.save(open('../lda_features/train.npy', 'wb'), train_lda_matrix)
np.save(open('../lda_features/test.npy', 'wb'), test_lda_matrix)